# Metadata Refresh for SQL Server (Project Setup)

This notebook contains the SQL Set-up scripts for the eltSnap project "Metadata Refresh for SQL Server"

These script needs to be run **after** ingesting the notebook called **Ingest ADS Notebook Files (Project)**

> To run these SQL scripts, first connect to your SQL Server instance of **eltsnap_v2**

## Add a Connection for the SQL Server where you want to Extract Metadata (Source)

Verify the Parameter Values in the EXEC statement below. You may need to _replace_ 'localhost' with your _server_name_

> Note: Initial Parameter Values can be substituted at runtime

In [3]:
EXEC [elt].[Save Connection by Name] 
     @connection_name = 'sql_server'
   , @server_name = 'localhost'
   , @database_name = 'master'
   , @provider = 'SQL Server Native Client 11.0'

(1 row affected)

(1 row affected)

(13 rows affected)

(0 rows affected)

(1 row affected)

Total execution time: 00:00:00.007

RETURN_CODE
1


## Import the "Notebook Based" Project into eltSnap

This will refresh the cooresponding logic used by eltSnap based on the _Ingested_ Notebook code

In [4]:

EXEC [ads].[Import Project from ADS Notebooks] 'Metadata Refresh v2', 'notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb';

Created/Updated: Insert Metadata Server Name

Created/Updated: Truncate STG tables

Created/Updated: Refresh Database Names

Created/Updated: Merge stg to meta schema - dim_database

Created/Updated: src to stg - SQL Server tables

...

computer_name,full_name,cell_name,sequence_number,source_node,pattern,src_connection_name,dst_connection_name,dst_schema,dst_table,truncate_dst,identity_insert,batch_size,source,executable,working_directory
NULL,notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb,Merge stg to meta schema - dim_column,16,$.cells.16,Execute SQL,eltsnap_v2,NULL,NULL,NULL,NULL,NULL,NULL,"MERGE [meta].[dim_column] AS DST USING [stg].[dim_column] AS SRC ON src.[server_name] = dst.[server_name] AND src.[database_name] = dst.[database_name] AND src.[table_schema] = dst.[table_schema] AND src.[table_name] = dst.[table_name] AND src.[column_name] = dst.[column_name] WHEN MATCHED AND ( DST.[ordinal_position] != SRC.[ordinal_position] OR DST.[is_nullable] != SRC.[is_nullable] OR DST.[data_type] != SRC.[data_type] OR DST.[char_max_length] != SRC.[char_max_length] OR DST.[primary_key] != SRC.[primary_key] ) THEN UPDATE SET [ordinal_position] = SRC.[ordinal_position] , [is_nullable] = SRC.[is_nullable] , [data_type] = SRC.[data_type] , [char_max_length] = SRC.[char_max_length] , [primary_key] = SRC.[primary_key] WHEN NOT MATCHED THEN INSERT ( [server_name] , [database_name] , [table_schema] , [table_name] , [column_name] , [ordinal_position] , [is_nullable] , [data_type] , [char_max_length] , [primary_key] ) VALUES ( [server_name] , [database_name] , [table_schema] , [table_name] , [column_name] , [ordinal_position] , [is_nullable] , [data_type] , [char_max_length] , [primary_key] ) WHEN NOT MATCHED BY SOURCE AND DST.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/ AND DST.[database_name] = /*${sql_server_Database}*/'eltsnap_v2'/**/ THEN DELETE; SELECT @@ROWCOUNT AS [row_count];",NULL,NULL


computer_name,full_name,cell_name,sequence_number,source_node,pattern,src_connection_name,dst_connection_name,dst_schema,dst_table,truncate_dst,identity_insert,batch_size,source,executable,working_directory
NULL,notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb,Merge stg to dbo schema - dim_table,18,$.cells.18,Execute SQL,eltsnap_v2,NULL,NULL,NULL,NULL,NULL,NULL,"EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_table', 'dbo', 'dim_table', 'Standard', 'server_name,database_name,table_schema,table_name','table_type';",NULL,NULL


computer_name,full_name,cell_name,sequence_number,source_node,pattern,src_connection_name,dst_connection_name,dst_schema,dst_table,truncate_dst,identity_insert,batch_size,source,executable,working_directory
NULL,notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb,Merge stg to dbo schema - dim_column,20,$.cells.20,Execute SQL,eltsnap_v2,NULL,NULL,NULL,NULL,NULL,NULL,"EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_column', 'dbo', 'dim_column', 'Standard', 'server_name,database_name,table_schema,table_name,column_name','is_nullable,data_type,char_max_length';",NULL,NULL


computer_name,full_name,cell_name,sequence_number,source_node,pattern,src_connection_name,dst_connection_name,dst_schema,dst_table,truncate_dst,identity_insert,batch_size,source,executable,working_directory
NULL,notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb,dim delete - dbo - dim_table,22,$.cells.22,Execute SQL,eltsnap_v2,NULL,NULL,NULL,NULL,NULL,NULL,"WITH [deleted_rows] AS ( SELECT [server_name] , [database_name] , [table_schema] , [table_name] FROM [dbo].[dim_table] EXCEPT SELECT [server_name] , [database_name] , [table_schema] , [table_name] FROM [meta].[dim_table] ) UPDATE [dbo].[dim_table] SET [row_expiration_date] = CAST(GETDATE() AS DATE) , [row_update_date] = CAST(GETDATE() AS DATE) FROM [dbo].[dim_table] t JOIN [deleted_rows] d ON d.[server_name] = t.[server_name] AND d.[database_name] = t.[database_name] AND d.[table_schema] = t.[table_schema] AND d.[table_name] = t.[table_name] WHERE t.[row_expiration_date] = '9999-12-31'; SELECT @@ROWCOUNT AS [row_count];",NULL,NULL


computer_name,full_name,cell_name,sequence_number,source_node,pattern,src_connection_name,dst_connection_name,dst_schema,dst_table,truncate_dst,identity_insert,batch_size,source,executable,working_directory
NULL,notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb,dim delete - dbo - dim_column,24,$.cells.24,Execute SQL,eltsnap_v2,NULL,NULL,NULL,NULL,NULL,NULL,"WITH [deleted_rows] AS ( SELECT [server_name] , [database_name] , [table_schema] , [table_name] , [column_name] FROM [dbo].[dim_column] EXCEPT SELECT [server_name] , [database_name] , [table_schema] , [table_name] , [column_name] FROM [meta].[dim_column] ) UPDATE [dbo].[dim_column] SET [row_expiration_date] = CAST(GETDATE() AS DATE) , [row_update_date] = CAST(GETDATE() AS DATE) FROM [dbo].[dim_column] c JOIN [deleted_rows] d ON d.[server_name] = c.[server_name] AND d.[database_name] = c.[database_name] AND d.[table_schema] = c.[table_schema] AND d.[table_name] = c.[table_name] AND d.[column_name] = c.[column_name] WHERE c.[row_expiration_date] = '9999-12-31'; SELECT @@ROWCOUNT AS [row_count];",NULL,NULL
